# Compute the readout fidelity of all qubits

In [ ]:
import numpy as np
from qibo.gates import M, X
from qibo.models import Circuit
from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

api = API(ConnectionConfiguration(username="qat", api_key="meow"))
api.select_device_id(9)

We first generate and queue the circuits we need to execute:

In [ ]:
circuits = []
qubits = np.arange(5)
for qubit in qubits:
    circuit0 = Circuit(5)
    circuit0.add(M(qubit))

    circuit1 = Circuit(5)
    circuit1.add(X(qubit))
    circuit1.add(M(qubit))
    
    circuits.extend([circuit0, circuit1])

results_id = api.execute(circuits, nshots=1000)

print(f"The id of the executed job is: {results_id}")

Once the jobs have been executed, we can use the cell below to compute the readout fidelities:

In [ ]:
results = api.get_results(results_id)
for qubit_idx in qubits:
    # We take the pair of results that correspond to the specific qubit
    results0 = results[qubit_idx * 2]
    results1 = results[qubit_idx * 2 + 1]

    clf_0 = (1 - results0["probabilities"]["0"] + results0["probabilities"]["1"]) / 2
    clf_1 = (1 - results1["probabilities"]["0"] + results1["probabilities"]["1"]) / 2
    fid_avg = (1 - clf_0 + clf_1) / 2
    print(f"### QUBIT: {qubit_idx} ###")
    print(f"Ground state: {results0}")
    print(f"Excited state: {results1}")
    print("Fidelity", fid_avg)
